# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Doi  ->  K. Doi  |  ['K. Doi']
M. Ramirez  ->  M. Ramirez-Tannus  |  ['M. Ramirez']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']


F. Walter  ->  F. Walter  |  ['F. Walter']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 72 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.05872


extracting tarball to tmp_2503.05872...

 done.


Found 150 bibliographic references in tmp_2503.05872/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.05909


...

 done.


Found 152 bibliographic references in tmp_2503.07074/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.07484


extracting tarball to tmp_2503.07484...

 done.


Found 128 bibliographic references in tmp_2503.07484/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.05872-b31b1b.svg)](https://arxiv.org/abs/2503.05872) | **Turbulence in protoplanetary disks: A systematic analysis of dust settling in 33 disks**  |
|| M. Villenave, et al. -- incl., <mark>K. Doi</mark> |
|*Appeared on*| *2025-03-11*|
|*Comments*| *Accepted in A&A*|
|**Abstract**|            The level of dust vertical settling and radial dust concentration in disks is of critical importance for understanding the efficiency of planet formation. We present the first uniform analysis of the vertical extent of millimeter dust for a representative sample of 33disks. We used radiative transfer modeling of archival high-angular-resolution (<=0.1") ALMA dust observations of inclined and ringed disks to estimate their vertical dust scale height, which was compared to estimated gas scale heights to characterize the level of vertical sedimentation. In all 23systems for which constraints could be obtained, we find that the outer parts of the disks are vertically settled. 5disks allow for the characterization of the dust scale height both within and outside approximately half the dust disk radius, showing a lower limit on their dust heights at smaller radii. This implies that the ratio between vertical turbulence and the Stokes number, $\alpha_z/\St$, decreases radially in these sources. For 21rings in 15disks, we also constrained the level of radial concentration of the dust, finding that about half of the rings are compatible with strong radial trapping. In most of these rings, vertical turbulence is found to be comparable to or weaker than radial turbulence, which is incompatible with the turbulence generated by the vertical shear instability at these locations. We further used our dust settling constraints to estimate the turbulence level under the assumption that the dust size is limited by fragmentation, finding typical upper limits around $\alpha_\text{frag}\leq10^{-3}$. In a few sources, we find that turbulence cannot be the main source of accretion. In the context of pebble accretion, we identify several disk regions that have upper limits on their dust concentration that would allow core formation to proceed efficiently, even at wide orbital distances outside of 50au.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.07074-b31b1b.svg)](https://arxiv.org/abs/2503.07074) | **JWST ASPIRE: How Did Galaxies Complete Reionization? Evidence for Excess IGM Transmission around ${\rm [O\,{\scriptstyle III}]}$ Emitters during Reionization**  |
|| K. Kakiichi, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2025-03-11*|
|*Comments*| *31 pages, 22 figures, submitted to the Open Journal of Astrophysics*|
|**Abstract**|            The spatial correlation between galaxies and the Ly$\alpha$ forest provides insights into how galaxies reionized the Universe. Here, we present initial results on the spatial cross-correlation between [OIII] emitters and Ly$\alpha$ forest at 5.4<z<6.5 from the JWST ASPIRE NIRCam/F356W Grism Spectroscopic Survey in z>6.5 QSO fields. Using data from five QSO fields, we find $2\sigma$ evidence for excess Ly$\alpha$ forest transmission at ~20-40 cMpc around [OIII] emitters at z=5.86, indicating that [OIII] emitters reside within a highly ionized IGM. At smaller scales, the Ly$\alpha$ forest is preferentially absorbed, suggesting gas overdensities around [OIII] emitters. Comparing with models including THESAN simulations, we interpret the observed cross-correlation as evidence for significant large-scale fluctuations of the IGM and the late end of reionization at z<6, characterized by ionized bubbles over 50 cMpc around [OIII] emitters. The required UV background necessitates an unseen population of faint galaxies around the [OIII] emitters. Furthermore, we find that the number of observed [OIII] emitters near individual transmission spikes is insufficient to sustain reionization in their surroundings, even assuming all [OIII] emitters harbour AGN with 100 % LyC escape fractions. Despite broad agreement, a careful analysis of ASPIRE and THESAN, using the observed host halo mass from the clustering of [OIII] emitters, suggests that the simulations underpredict the observed excess IGM transmission around [OIII] emitters, challenging our model of reionization. Potential solutions include larger ionized bubbles at z<6, more enhanced large-scale UV background or temperature fluctuations of the IGM, and possibly a patchy early onset of reionization at z>10. Current observational errors are dominated by cosmic variance, meaning future analyses of more QSO fields from JWST will improve the results.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.07484-b31b1b.svg)](https://arxiv.org/abs/2503.07484) | **Euclid: Early Release Observations -- The Intracluster Light of Abell 2390**  |
|| A. Ellien, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-11*|
|*Comments*| *23 pages, 20 figures, 2 tables submitted to A\&A*|
|**Abstract**|            Intracluster light (ICL) provides a record of the dynamical interactions undergone by clusters, giving clues on cluster formation and evolution. Here, we analyse the properties of ICL in the massive cluster Abell 2390 at redshift z=0.228. Our analysis is based on the deep images obtained by the Euclid mission as part of the Early Release Observations in the near-infrared (Y, J, H bands), using the NISP instrument in a 0.75 deg$^2$ field. We subtracted a point--spread function (PSF) model and removed the Galactic cirrus contribution in each band after modelling it with the DAWIS software. We then applied three methods to detect, characterise, and model the ICL and the brightest cluster galaxy (BCG): the CICLE 2D multi-galaxy fitting; the DAWIS wavelet-based multiscale software; and a mask-based 1D profile fitting. We detect ICL out to 600 kpc. The ICL fractions derived by our three methods range between 18% and 36% (average of 24%), while the BCG+ICL fractions are between 21% and 41% (average of 29%), depending on the band and method. A galaxy density map based on 219 selected cluster members shows a strong cluster substructure to the south-east and a smaller feature to the north-west. Based on colours, the ICL (out to about 400 kpc) seems to be built by the accretion of small systems (M ~ $10^{9.5}$ solar mass), or from stars coming from the outskirts of Milky Way-type galaxies (M ~ $10^{10}$ solar mass). Though Abell 2390 does not seem to be undergoing a merger, it is not yet fully relaxed, since it has accreted two groups that have not fully merged with the cluster core. We estimate that the contributions to the inner 300 kpc of the ICL of the north-west and south-east subgroups are 21% and 9% respectively.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.05909-b31b1b.svg)](https://arxiv.org/abs/2503.05909) | **Early Light Curve Excess in Type IIb Supernovae Observed by the ATLAS Survey: Qualitative Constraints on Progenitor Systems**  |
|| B. Ayala, et al. -- incl., <mark>M. Ramirez</mark> |
|*Appeared on*| *2025-03-11*|
|*Comments*| *27 pages, 18 figures. Submitted to Astronomy & Astrophysics, under review*|
|**Abstract**|            Type IIb supernovae (SNe IIb) often exhibit an early light curve excess (EE) preceding the main peak powered by radioactive nickel decay. The physical origin of this early emission remains an open question. Among the proposed scenarios, shock cooling emission-resulting from the interaction between the shockwave and extended envelopes-is considered the most plausible mechanism. The frequency of these events remains unconstrained. This study aims to quantify the frequency of EE in SNe IIb and investigate its physical origin by analyzing optical light curves from the Asteroid Terrestrial-impact Last Alert System (ATLAS) survey. We selected 74 SNe IIb from 153 spectroscopically classified events in the Transient Name Server (TNS) database, observed by ATLAS, with peak fluxes exceeding 150 {\mu}Jy and explosion epoch uncertainties lower than six days. Using light curve model fitting and outlier analysis, we identified SNe IIb exhibiting EE and analyzed their photometric properties. We found 21 SNe IIb with EE, corresponding to a frequency of approximately 28-40%, with the higher value obtained under the most stringent data cuts. The EE's duration and color evolution are consistent with shock cooling in extended hydrogen-rich envelopes. We also found that EE SNe IIb have longer rise times and faster post-peak decline rates than non-EE SNe IIb, while both groups share similar peak absolute magnitudes. Our findings suggest that EE and non-EE SNe IIb likely share similar initial progenitor masses but differ in ejecta mass properties, potentially due to varying degrees of binary interaction. This study provides constraints on the evolutionary pathways of SNe IIb progenitors as compact stars with and without extended hydrogen envelopes.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.05872/./figures/HD163296_LkCa15_V1094Sco_UpperLowerLimit_constraints.png', 'tmp_2503.05872/./figures/DoAr25_HD142666_UpperLowerLimit_constraints.png', 'tmp_2503.05872/./figures/alphaZ_over_alphaR.png']
copying  tmp_2503.05872/./figures/HD163296_LkCa15_V1094Sco_UpperLowerLimit_constraints.png to _build/html/
copying  tmp_2503.05872/./figures/DoAr25_HD142666_UpperLowerLimit_constraints.png to _build/html/
copying  tmp_2503.05872/./figures/alphaZ_over_alphaR.png to _build/html/
exported in  _build/html/2503.05872.md
    + _build/html/tmp_2503.05872/./figures/HD163296_LkCa15_V1094Sco_UpperLowerLimit_constraints.png
    + _build/html/tmp_2503.05872/./figures/DoAr25_HD142666_UpperLowerLimit_constraints.png
    + _build/html/tmp_2503.05872/./figures/alphaZ_over_alphaR.png
found figures ['tmp_2503.07074/./figures/galaxy-IGM_cross-correlation_measurement_vs_RT_FRAMEWORK_full.png', 'tmp_2503.07074/./figures/ASPIRE_vs_THESAN_Mstar1e10_xHI_clean.png', 'tmp_2503.07074/./fig

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\St}{\text{St}}$
$\newcommand{\Sc}{\text{Sc}}$</div>



<div id="title">

# Turbulence in protoplanetary disks: A systematic analysis of dust settling in 33 disks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.05872-b31b1b.svg)](https://arxiv.org/abs/2503.05872)<mark>Appeared on: 2025-03-11</mark> -  _Accepted in A&A_

</div>
<div id="authors">

M. Villenave, et al. -- incl., <mark>K. Doi</mark>

</div>
<div id="abstract">

**Abstract:** The level of dust vertical settling and radial dust concentration in protoplanetary disks is of critical importance for understanding the efficiency of planet formation. Here, we present the first uniform analysis of the vertical extent of millimeter dust for a representative sample of 33  protoplanetary disks, covering broad ranges of disk evolutionary stages and stellar masses.We used radiative transfer modeling of archival high-angular-resolution ( $\lesssim0.1\arcsec$ ) ALMA dust observations of inclined and ringed disks to estimate their vertical dust scale height, which was compared to estimated gas scale heights to characterize the level of vertical sedimentation.In all 23 systems for which constraints could be obtained, we find that the outer parts of the disks are vertically settled.Five disks allow for the characterization of the dust scale height both within and outside approximately half the dust disk radius, showing a lower limit on their dust heights at smaller radii. This implies that the ratio between vertical turbulence, $\alpha_z$ , and the Stokes number, $\alpha_z/\St$ , decreases radially in these sources. For 21 rings in 15 disks, we also constrained the level of radial concentration of the dust, finding that about half of the rings are compatible with strong radial trapping. In most of these rings, vertical turbulence is found to be comparable to or weaker than radial turbulence,which is incompatible with the turbulence generated by the vertical shear instability at these locations. We further used our dust settling constraints to estimate the turbulence level under the assumption that the dust size is limited by fragmentation, finding typical upper limits around $\alpha_\text{frag}\lesssim10^{-3}$ .  In a few sources, we find that turbulence cannot be the main source of accretion.Finally, in the context of pebble accretion, we identify several disk regions that have upper limits on their dust concentrationthat would allow core formation to proceed efficiently, even at wide orbital distances outside of 50 au.

</div>

<div id="div_fig1">

<img src="tmp_2503.05872/./figures/HD163296_LkCa15_V1094Sco_UpperLowerLimit_constraints.png" alt="Fig7" width="100%"/>

**Figure 7. -** Vertically thick inner ring and thin outer disk in HD163296, LkCa 15, and V1094Sco. The shaded colors on the major axis cuts indicate the location of the azimuthal profiles (orange) and rings of interest (yellow and green).
    "Flat" models are too thin vertically to show sufficient azimuthal variation compared to the data (third panels), while "Thick" models  are too vertically extended to reproduce the outer disk/ring brightness in all three disks. The beam size and a 25 au scale are indicated in the bottom left corner of the first panels. (*fig:upperLower_LkCa15_V1094Sco*)

</div>
<div id="div_fig2">

<img src="tmp_2503.05872/./figures/DoAr25_HD142666_UpperLowerLimit_constraints.png" alt="Fig8" width="100%"/>

**Figure 8. -** Resolved thickness in the inner disk half and flat outer disk in DoAr 25 and HD 142666. For both disks, "Flat" models show too strong of a gap in the inner region (third panels), indicating that the disks must be thicker. On the other hand, "Moderate" and "Thick" models do not reproduce the gap and ring in DoAr 25, and the outer disk profile in both disks (last panels), highlighting the presence of a flat outer disk. The beam size and a 25 au scale are indicated in the bottom left corner of the first panels. (*fig:upperLower_DoAr25_HD142666*)

</div>
<div id="div_fig3">

<img src="tmp_2503.05872/./figures/alphaZ_over_alphaR.png" alt="Fig9" width="100%"/>

**Figure 9. -** Vertical $\alpha_z/$\St$$(top), radial $\alpha_r/$\St$$(middle), and their ratio $\alpha_z/\alpha_r$(bottom). In the bottom panel, we mark the rings without a full inclusion into the domain of the vertical constraints with a lighter blue. In all panels, constraints obtained by previous studies are shown in gray. The labels correspond to the constrained regions in \autoref{tab:verticalTrapping}. The underscore `\_1' corresponds to the only or first region with a constraint in a disk, while `\_2' displays the results for the second region. (*fig:radial_vertical_alpha*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.05872"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\HI}{{\rm H {\scriptstyle I}}}$
$\newcommand{\HII}{{\rm H {\scriptstyle II}}}$
$\newcommand{\HeI}{{\rm He {\scriptstyle I}}}$
$\newcommand{\HeII}{{\rm He {\scriptstyle II}}}$
$\newcommand{\HeIII}{{\rm He {\scriptstyle III}}}$
$\newcommand{\CII}{{\rm C {\scriptstyle II}}}$
$\newcommand{\CIII}{{\rm C {\scriptstyle III}}}$
$\newcommand{\CIV}{{\rm C {\scriptstyle IV}}}$
$\newcommand{\SiII}{{\rm Si {\scriptstyle II}}}$
$\newcommand{\SiIII}{{\rm Si {\scriptstyle III}}}$
$\newcommand{\MgII}{{\rm Mg {\scriptstyle II}}}$
$\newcommand{\NII}{{\rm N {\scriptstyle II}}}$
$\newcommand{\NI}{{\rm N {\scriptstyle I}}}$
$\newcommand{\NV}{{\rm N {\scriptstyle V}}}$
$\newcommand{\NVI}{{\rm N {\scriptstyle VI}}}$
$\newcommand{\OI}{{\rm O {\scriptstyle I}}}$
$\newcommand{\OII}{{\rm[O {\scriptstyle II}]}}$
$\newcommand{\OIII}{{\rm[O {\scriptstyle III}]}}$
$\newcommand{\OVI}{{\rm O {\scriptstyle VI}}}$
$\newcommand{\SII}{{\rm S {\scriptstyle II}}}$
$\newcommand{\rmnum}[1]{\romannumeral #1}$
$\newcommand{\Rmnum}[1]{\expandafter\@slowromancap\romannumeral #1@}$
$\newcommand{\nHI}{n_{\rm HI}}$
$\newcommand{\xHI}{x_{\rm HI}}$
$\newcommand{\xHII}{x_{\mbox{\tiny H\Rmnum{2}}}}$
$\newcommand{\eHI}{\epsilon_{\mbox{\tiny H\Rmnum{1}}}}$
$\newcommand{\LyA}{\mbox{Ly}\alpha}$
$\newcommand{\NHI}{N_{\mbox{\tiny H\Rmnum{1}}}}$
$\newcommand{\fHI}{\langle f_{\mbox{\tiny H\Rmnum{1}}}\rangle}$
$\newcommand{\fHII}{\langle f_{\mbox{\tiny H\Rmnum{2}}}\rangle}$
$\newcommand{\CDDF}{\frac{\partial^2\mathcal{N}}{\partial\NHI\partial z}}$
$\newcommand{\sigmaHI}{\sigma_{\mbox{\tiny H\Rmnum{1}}}}$
$\newcommand{\Muv}{M_{\mbox{\tiny UV}}}$
$\newcommand{\br}{\boldsymbol{r}}$
$\newcommand{\A}{\mbox{Å}}$
$\newcommand{\TIGM}{T_{\rm IGM}}$
$\newcommand{\TIGMi}{T_{{\rm IGM},i}}$
$\newcommand{\TIGMr}{\langle\overline{T}_{\rm IGM}(r_\perp)\rangle}$
$\newcommand{\Mh}{M_{\rm h}}$
$\newcommand{\comment}{\textcolor{red}}$</div>



<div id="title">

# $\vspace{-0.5cm}$JWST ASPIRE: How Did Galaxies Complete Reionization? \ Evidence for Excess IGM Transmission around $\OIII$ Emitters during Reionization$\vspace{-15mm}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.07074-b31b1b.svg)](https://arxiv.org/abs/2503.07074)<mark>Appeared on: 2025-03-11</mark> -  _31 pages, 22 figures, submitted to the Open Journal of Astrophysics_

</div>
<div id="authors">

K. Kakiichi, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** The spatial correlation between galaxies and the Ly $\alpha$ forest of the intergalactic medium (IGM) provides insights into how galaxies reionized the Universe. Here, we present initial results on the spatial cross-correlation between $\OIII$ emitters and Ly $\alpha$ forest transmission at $5.4<z<6.5$ from the JWST ASPIRE NIRCam/F356W Grism Spectroscopic Survey in $z>6.5$ QSO fields. Using data from five QSO fields, we find $2\sigma$ evidence for excess Ly $\alpha$ forest transmission at $\sim20-40 \rm cMpc$ around $\OIII$ emitters at $\langle z\rangle\simeq5.86$ , indicating that $\OIII$ emitters reside within a highly ionized IGM. At smaller scales, the Ly $\alpha$ forest is preferentially absorbed, suggesting gas overdensities around $\OIII$ emitters. Comparing with models, including THESAN cosmological radiation hydrodynamic simulations, we interpret the observed cross-correlation as evidence for significant large-scale fluctuations of the IGM and the late end of reionization at $z<6$ , characterized by ionized bubbles over $50\rm cMpc$ around $\OIII$ emitters. The required UV background necessitates an unseen population of faint galaxies around the $\OIII$ emitters with average LyC leakage of $\log_{10} \langle f_{\text{esc}} \xi_{\text{ion}} \rangle / [{\text{erg}^{-1} \text{Hz}}] \simeq 24.5$ down to $M_{\text{UV}} = -10$ . Furthermore, we find that the number of observed $\OIII$ emitters near individual transmission spikes is insufficient to sustain reionization in their surroundings, even assuming all $\OIII$ emitters harbour AGN with $100 \%$ LyC escape fractions. Despite broad agreement, a careful analysis of ASPIRE and THESAN, using the observed host halo mass from the clustering of $\OIII$ emitters, suggests that the simulations underpredict the observed excess IGM transmission around $\OIII$ emitters, challenging our model of reionization. Potential solutions include larger ionized bubbles at $z<6$ , further enhancement of large-scale UV background or temperature fluctuations of the IGM, and possibly a patchy early onset of reionization at $z>10$ . Current observational errors are dominated by cosmic variance, meaning future analyses of more QSO fields from JWST will improve the results.

</div>

<div id="div_fig1">

<img src="tmp_2503.07074/./figures/galaxy-IGM_cross-correlation_measurement_vs_RT_FRAMEWORK_full.png" alt="Fig16" width="100%"/>

**Figure 16. -** Comparison of the observed mean Ly$\alpha$ forest transmission around $\OIII$ emitters at $\langle z\rangle=5.86$ with the theoretical model based on analytic radiative transfer + CLF framework  ([ and Kakiichi 2018](https://ui.adsabs.harvard.edu/abs/2018MNRAS.479...43K)) . The top panels show the average photoionization rate around $\OIII$ emitters, $\langle \Gamma_{\rm HI}(r) \rangle$(solid line: contribution from surrounding galaxies; dashed line: central $\OIII$ emitters) on the left y-axis. The average gas overdensity profile around $\OIII$ emitters is indicated on the right y-axis. The bottom panels indicate the mean Ly$\alpha$ forest transmission around $\OIII$ emitters, $\langle T_{\rm IGM}(r)\rangle$ as a function of radial distance from $\OIII$ emitters. (Left): The model prediction with varying average LyC leakage $\langle f_{\rm esc}\xi_{\rm ion}\rangle=24.4,24.5,24.6$. The other parameters are fixed to the fiducial values as indicated in the text. (Middle): The model prediction with varying relative contribution from bright vs faint galaxies (see text). (Right): The model prediction with varying mean free path $\lambda_{\rm mfp}=1.0,2.0,3.0\rm pMpc$. The black squares show the observed mean Ly$\alpha$ forest transmission around $\OIII$ emitters with $1\sigma$ error estimated from the Jackknife method. (*fig:model*)

</div>
<div id="div_fig2">

<img src="tmp_2503.07074/./figures/ASPIRE_vs_THESAN_Mstar1e10_xHI_clean.png" alt="Fig17" width="100%"/>

**Figure 17. -** Comparison of the observed $\OIII$ emitter-Ly$\alpha$ forest cross-correlation, $\langle T_{\rm IGM}(r)\rangle/\overline{T}_{\rm IGM}-1$, at $\langle z\rangle=5.86$(black squares) with the results from the THESAN cosmological radiation hydrodynamic simulation. The coloured curves show the results form THESAN-1 snapshots from $z=6.2$ to $5.5$ corresponding to global $\HI$ fractions of $\langle$\xHI$\rangle=0.14$ to $3.4\times10^{-3}$. We chose the central galaxies with stellar mass of $M_\ast>10^{10} \rm M_\odot$ from the THESAN simulation. The black errorbars show the $1\sigma$ error estimated from the Jackknife method. (*fig:thesan*)

</div>
<div id="div_fig3">

<img src="tmp_2503.07074/./figures/THESAN_map_full.png" alt="Fig18" width="100%"/>

**Figure 18. -** Sliced maps of the $\HI$ number density $n_{$\HI$}$(left), the photoionization rate $\Gamma_{\rm HI}$(middle), and the temperature $T$(right) around a central galaxy (black star symbol) with a stellar mass of $M_\ast>10^{10} \rm M_\odot$ in the THESAN-1 snapshot at $z=5.83$, corresponding to the fourth bluest galaxy-Ly$\alpha$ forest cross-correlation in Figure \ref{fig:thesan}. The open circles show the distribution of surrounding galaxies with stellar masses of $M_\ast>5\times10^7 \rm M_\odot$. The large dashed circle indicates a radius of $40\rm cMpc$ around the central galaxy. All sliced maps have a width of $3.7 \rm cMpc$. (*fig:thesan_map*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.07074"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mireia}[1]{{\color{purple1}\textsf{#1}}}$
$\newcommand{\amael}[1]{{\color{brown}\textsf{#1}}}$
$\newcommand{\jesse}[1]{{\color{green}\textsf{#1}}}$
$\newcommand{\paola}[1]{{\color{orange}\textsf{#1}}}$
$\newcommand{\lammim}[1]{{\color{magenta}\textsf{#1}}}$
$\newcommand{\florence}[1]{{\color{cyan}\textsf{#1}}}$
$\newcommand{\yoli}[1]{{\color{pink}\textsf{#1}}}$
$\newcommand{\noisechisel}{\texttt{NoiseChisel}}$
$\newcommand{\sextractor}{\texttt{SExtractor}}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$</div>



<div id="title">

# $\Euclid$: Early Release Observations -  The intracluster light of Abell 2390 $\thanks{This paper is published on behalf of the Euclid Consortium.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.07484-b31b1b.svg)](https://arxiv.org/abs/2503.07484)<mark>Appeared on: 2025-03-11</mark> -  _23 pages, 20 figures, 2 tables submitted to A\&A_

</div>
<div id="authors">

A. Ellien, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Intracluster light (ICL) provides a record of the dynamical interactions undergone by clusters, giving clues on cluster formation and evolution. Here, we analyse the properties of ICL in the massive cluster Abell 2390 at redshift $z= 0.228$ .Our analysis is based on the deep images obtained by the $\Euclid$ mission as part of the Early Release Observations in the near-infrared ( $\YE$ , $\JE$ , $\HE$ bands), using the NISP instrument in a 0.75 deg $^2$ field. We subtracted a point--spread function (PSF) model and removed the Galactic cirrus contribution in each band after modelling it with the \texttt{DAWIS} software. We then applied three methods to detect, characterise, and model the ICL and the brightest cluster galaxy (BCG): the \texttt{CICLE} 2D multi-galaxy fitting; the \texttt{DAWIS} wavelet-based multiscale software; and a mask-based 1D profile fitting.We detect ICL out to 600 kpc. The ICL fractions derived by our three methods range between 18 \% and 36 \% (average of 24 \% ), while the BCG+ICL fractions are between 21 \% and 41 \% (average of 29 \% ), depending on the band and method. A galaxy density map based on 219 selected cluster members shows a strong cluster substructure to the south-east and a smaller feature to the north-west.Ellipticals dominate the cluster's central region, with a centroid offset from the BCG by about 70 kpc and distribution following that of the ICL, while spirals do not trace the entire ICL but rather substructures. The comparison of the BCG+ICL, mass from gravitational lensing, and X-ray maps show that the BCG+ICL is the best tracer of substructures in the cluster.Based on colours, the ICL (out to about 400 kpc) seems to be built by the accretion of small systems ( $M \sim 10^{9.5}\si{\solarmass}$ ), or from stars coming from the outskirts of Milky Way-type galaxies ( $M \sim 10^{10}\si{\solarmass}$ ). Though Abell 2390 does not seem to be undergoing a merger, it is not yet fully relaxed, since it has accreted two groups that have not fully merged with the cluster core. We estimate that the contributions to the inner 300 kpc of the ICL of the north-west and south-east subgroups are 21 \% and 9 \% respectively.

</div>

<div id="div_fig1">

<img src="tmp_2503.07484/./figures/Contours_allspec_zphot_all_galaxies_v4.png" alt="Fig11" width="100%"/>

**Figure 11. -** Contour maps of the spatial distribution of elliptical (red) and spiral (blue) galaxies overlaid on the \HE-band image of A 2390 in the upper panel, and the \texttt{DAWIS} BCG+ICL map in the lower panel. The figure shows a region of $\ang{;;700}\times\ang{;;700}$($2.5 {\rm Mpc}\times2.5 {\rm Mpc}$) centred on the BCG. (*fig:morph_contours*)

</div>
<div id="div_fig2">

<img src="tmp_2503.07484/./figures/rgb_Yoli_wgroups.png" alt="Fig18.1" width="50%"/><img src="tmp_2503.07484/./figures/rgb_Amael_wgroups.png" alt="Fig18.2" width="50%"/>

**Figure 18. -** RGB images of the BCG+ICL in the core of A 2390 ($700 $\arcsec$\times 700 $\arcsec$$). The left panel shows the \texttt{CICLE} RGB map and the right panel the \texttt{DAWIS} RGB map. The numbers in both maps mark the position of the galaxies in the groups identified by DS+ (see Sect. \ref{sec:substructures}), listed in Table \ref{table:DS+}. Both colour images were generated with Trilogy  ([Coe, Umetsu and Zitrin 2012]()) . (*fig:rgb_groups*)

</div>
<div id="div_fig3">

<img src="tmp_2503.07484/./figures/A2390_allz_galdens.png" alt="Fig10" width="100%"/>

**Figure 10. -** Density map of the 219 galaxies selected as belonging to the cluster A 2390 (see main text). The green contour levels correspond to 3$\sigma$, 10$\sigma$, and 20$\sigma$ above the background.  North is up and east is to the left. The image size is  $\ang{0.4;;} \times \ang{0.4;;}$(5.24 Mpc $\times$ 5.24 Mpc) and is centred on the BCG.  (*fig:densmap*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.07484"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

359  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
